In [7]:
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import s3fs
import re
from pelicanfs.core import PelicanFileSystem, PelicanMap,OSDFFileSystem 
import fsspec.implementations.http as fshttp
import geopandas as gpd
from sentinelhub import SHConfig, BBox, SentinelHubRequest, DataCollection, CRS

/glade/u/home/harshah/.conda/envs/osdf/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import dask 
from dask_jobqueue import PBSCluster
from dask.distributed import Client
from dask.distributed import performance_report

In [9]:
# Load the GeoJSON file
geojson_path = '/glade/u/home/harshah/osdf_examples/3100180240.geojson' 
gdf = gpd.read_file(geojson_path)

# Display the loaded GeoDataFrame
print(gdf)

                     id  SUB_AREA  COAST     PFAF_ID  DIST_MAIN    HYBAS_ID  \
0  00140000000000002983     128.1      0  3512704524      784.3  3100180240   

   DIST_SINK   NEXT_DOWN  ORDER  ENDO    MAIN_BAS   NEXT_SINK   SORT  UP_AREA  \
0      784.3  3100180230      4     0  3100009670  3100009670  66318    128.1   

                                            geometry  
0  POLYGON ((134.51667 66.87917, 134.51752 66.875...  


In [6]:
# Extract AOI geometry
aoi_geometry = gdf.geometry.iloc[0]

# Get AOI centroid for visualization
centroid  = aoi_geometry.centroid
long, lat = centroid.x, centroid.y

In [8]:
# SentinelHub configuration (make sure you have configured credentials)
config = SHConfig()

# Define bounding box from AOI geometry (using WGS84 CRS)
bbox = BBox(bbox=aoi_geometry.bounds, crs=CRS.WGS84)

# Set up the request for Sentinel-2 data
request = SentinelHubRequest(
    data_collection=DataCollection.SENTINEL2_L2A,
    bbox=bbox,
    resolution=(10, 10),  # 10m resolution
    time_interval=('2019-06-01', '2023-09-30'),  # Time range: June to September for years 2019-2023
    maxcc=0.05,  # Maximum cloud coverage (5%)
    bands=['B04', 'B08', 'SCL'],  # Red, NIR, and Scene Classification Layer for cloud masking
    config=config
)

# Fetch the data
response = request.get_data()

TypeError: SentinelHubRequest.__init__() missing 3 required positional arguments: 'evalscript', 'input_data', and 'responses'